Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [76]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [37]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.

In [38]:
using AmbulanceDeployment

┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278


In [39]:
using DataFrames, JLD, Distributions, CSV, Random, JSON, DelimitedFiles, Dates #Plots I don't think we use Plots
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

In [41]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [104]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 45
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3199
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
#CALLS IN MINUTES #test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame  
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]#convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

n ambulances 45
 locations  44
 nregions 3199
 coverage (3233, 44)
 demand (0, 2)
 adjacency (3199, 3199)running 45 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [105]:
x = amb_deployment[model_dict[model_name]][namb]#convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

44-element Array{Int64,1}:
 1
 3
 1
 1
 0
 0
 1
 2
 1
 0
 0
 0
 0
 ⋮
 1
 2
 0
 1
 0
 0
 1
 0
 1
 0
 2
 1

In [61]:
round.(Int, xnew)

44×1 Array{Int64,2}:
 0
 1
 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 ⋮
 1
 1
 2
 0
 0
 1
 1
 0
 1
 1
 1
 1

###### problem.available

In [43]:
function logRegression(gridTime)
    if(gridTime < 0)
        print("grid time less than 0: ", gridTime)
    end
    gridTime = gridTime / 60
#         slope = 0.23868463365149437
#         intercept = 1.261777659720721
    slope= 0.4011478655815724 #test calls
    intercept= 1.0469242277715907 #test calls
    ax = log(gridTime)*slope
    new_time = exp(ax + intercept)
    new_time =  new_time * 60
    new_time
end

logRegression (generic function with 1 method)

In [44]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        
        travel_time = ceil(Int, problem.emergency_calls[id, Symbol("stn$(i)_min")]) #60*
        println(travel_time)
        travel_time = logRegression(travel_time) # gridTime2Bin(travel_time) #adjust time
        travel_time = ceil(travel_time) #time mod
        println(travel_time)
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
        println("and there are this many amb available: ", sum(problem.available))
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    #60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes why the hell would we add 15 minutes and then also the hopsital min!! smh
    conveytime = ceil(Int, problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    conveytime = ceil(logRegression(conveytime)/60) # minutes #adjust time
    #conveytime = 15
    ems.eventlog[id, :conveytime] = conveytime 
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    returntime = ceil(logRegression(returntime)) #adjust time
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [83]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 1012
closest available station : 13
327
338.0
Ambulance has arrived on the scene for event 1 at time 376
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 25
477
393.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance has arrived on the scene for event 3 at time 1066
calling event id: 5 time: 1166 value: 1621
closest available station : 25
465
389.0
calling event id: 6 time: 1187 value: 578
closest available station : 2
613
435.0
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
513.0
Ambulance has arrived on the scene for event 5 at time 1555
Ambulance for event 1 has arrived at the hospital at time 1574
calling event id: 8 time: 1578 value: 825
closest availabl

Ambulance for event 26 is returning to the station at 8170
Ambulance has arrived on the scene for event 27 at time 8229
Ambulance has arrived on the scene for event 29 at time 8371
Ambulance has arrived on the scene for event 30 at time 8397
Ambulance has arrived on the scene for event 28 at time 8443
calling event id: 31 time: 8588 value: 760
closest available station : 18
535
412.0
Ambulance from event 26 has returned to station 21
Ambulance from event 23 has returned to station 21
calling event id: 32 time: 8673 value: 1008
closest available station : 18
856
497.0
Ambulance for event 25 has arrived at the hospital at time 8761
Ambulance for event 25 is returning to the station at 8768
Ambulance has arrived on the scene for event 31 at time 9000
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
344.0
Ambulance has arrived on the scene for event 32 at time 9170
Ambulance for event 27 has arrived at the hospital at time 9196
Ambulance from event 25 has return

closest available station : 25
377
358.0
calling event id: 55 time: 15714 value: 1437
closest available station : 21
409
370.0
calling event id: 56 time: 15831 value: 1664
closest available station : 34
475
392.0
Ambulance has arrived on the scene for event 53 at time 15910
Ambulance for event 52 has arrived at the hospital at time 15949
Ambulance for event 52 is returning to the station at 15955
Ambulance from event 50 has returned to station 25
Ambulance has arrived on the scene for event 54 at time 16057
Ambulance has arrived on the scene for event 55 at time 16084
Ambulance from event 51 has returned to station 44
Ambulance has arrived on the scene for event 56 at time 16223
calling event id: 57 time: 16247 value: 1009
closest available station : 21
829
491.0
Ambulance from event 52 has returned to station 13
Ambulance for event 53 has arrived at the hospital at time 16662
Ambulance for event 53 is returning to the station at 16667
Ambulance has arrived on the scene for event 57 at

Ambulance for event 76 has arrived at the hospital at time 25851
Ambulance for event 76 is returning to the station at 25858
calling event id: 79 time: 25937 value: 702
closest available station : 13
672
451.0
Ambulance from event 77 has returned to station 21
Ambulance from event 78 has returned to station 21
Ambulance from event 76 has returned to station 25
Ambulance has arrived on the scene for event 79 at time 26388
calling event id: 80 time: 26570 value: 1625
closest available station : 25
463
388.0
calling event id: 81 time: 26618 value: 1257
closest available station : 21
395
365.0
calling event id: 82 time: 26633 value: 1568
closest available station : 42
230
294.0
calling event id: 83 time: 26778 value: 1380
closest available station : 4
254
305.0
Ambulance has arrived on the scene for event 82 at time 26927
Ambulance has arrived on the scene for event 80 at time 26958
Ambulance has arrived on the scene for event 81 at time 26983
Ambulance has arrived on the scene for event 8

Ambulance has arrived on the scene for event 104 at time 38310
calling event id: 106 time: 38341 value: 458
closest available station : 31
692
456.0
Ambulance has arrived on the scene for event 105 at time 38498
calling event id: 107 time: 38642 value: 1865
closest available station : 17
407
369.0
Ambulance for event 103 has arrived at the hospital at time 38737
calling event id: 108 time: 38739 value: 1624
closest available station : 25
357
350.0
Ambulance for event 103 is returning to the station at 38743
Ambulance has arrived on the scene for event 106 at time 38797
Ambulance has arrived on the scene for event 107 at time 39011
Ambulance for event 104 has arrived at the hospital at time 39013
Ambulance for event 104 is returning to the station at 39018
Ambulance from event 103 has returned to station 13
calling event id: 109 time: 39060 value: 1808
closest available station : 25
349
347.0
Ambulance has arrived on the scene for event 108 at time 39089
Ambulance has arrived on the sce

...

Ambulance from event 906 has returned to station 18
calling event id: 910 time: 255367 value: 1619
closest available station : 17
587
427.0
calling event id: 911 time: 255429 value: 1379
closest available station : 21
262
309.0
Ambulance for event 907 has arrived at the hospital at time 255440
Ambulance for event 907 is returning to the station at 255446
calling event id: 912 time: 255506 value: 1440
closest available station : 21
301
327.0
calling event id: 913 time: 255613 value: 886
closest available station : 13
512
404.0
calling event id: 914 time: 255645 value: 890
closest available station : 21
1029
535.0
Ambulance has arrived on the scene for event 911 at time 255738
Ambulance for event 909 has arrived at the hospital at time 255772
Ambulance for event 909 is returning to the station at 255782
Ambulance has arrived on the scene for event 910 at time 255794
Ambulance has arrived on the scene for event 912 at time 255833
Ambulance from event 907 has returned to station 18
Ambulan

611
434.0
Ambulance for event 928 has arrived at the hospital at time 261221
Ambulance has arrived on the scene for event 937 at time 261221
Ambulance for event 928 is returning to the station at 261231
Ambulance has arrived on the scene for event 936 at time 261246
Ambulance for event 933 has arrived at the hospital at time 261270
Ambulance for event 930 has arrived at the hospital at time 261270
Ambulance for event 933 is returning to the station at 261278
Ambulance for event 930 is returning to the station at 261280
Ambulance from event 929 has returned to station 26
Ambulance from event 927 has returned to station 21
Ambulance for event 932 has arrived at the hospital at time 261335
Ambulance for event 932 is returning to the station at 261341
Ambulance has arrived on the scene for event 938 at time 261445
calling event id: 940 time: 261500 value: 1315
closest available station : 21
480
394.0
calling event id: 941 time: 261510 value: 1562
closest available station : 25
643
443.0
ca

279.0
Ambulance from event 954 has returned to station 2
calling event id: 963 time: 266066 value: 2225
closest available station : 41
426
376.0
Ambulance has arrived on the scene for event 961 at time 266155
Ambulance has arrived on the scene for event 959 at time 266174
Ambulance has arrived on the scene for event 960 at time 266277
Ambulance has arrived on the scene for event 962 at time 266318
Ambulance from event 953 has returned to station 25
Ambulance from event 955 has returned to station 2
calling event id: 964 time: 266425 value: 1568
closest available station : 42
230
294.0
Ambulance has arrived on the scene for event 963 at time 266442
Ambulance for event 958 has arrived at the hospital at time 266683
Ambulance for event 958 is returning to the station at 266691
calling event id: 965 time: 266716 value: 642
closest available station : 27
852
496.0
Ambulance has arrived on the scene for event 964 at time 266719
Ambulance for event 956 has arrived at the hospital at time 2668

322
336.0
Ambulance for event 981 has arrived at the hospital at time 275916
Ambulance for event 981 is returning to the station at 275923
calling event id: 985 time: 276131 value: 1314
closest available station : 21
696
457.0
Ambulance for event 983 has arrived at the hospital at time 276192
Ambulance for event 983 is returning to the station at 276198
Ambulance has arrived on the scene for event 984 at time 276202
Ambulance from event 981 has returned to station 21
Ambulance for event 982 has arrived at the hospital at time 276469
Ambulance for event 982 is returning to the station at 276477
Ambulance has arrived on the scene for event 985 at time 276588
Ambulance from event 983 has returned to station 4
calling event id: 986 time: 276780 value: 1684
closest available station : 25
203
279.0
Ambulance for event 984 has arrived at the hospital at time 276937
Ambulance for event 984 is returning to the station at 276942
Ambulance from event 982 has returned to station 2
Ambulance has ar

(1000×11 DataFrame
  Row │ id     dispatch_from  waittime  responsetime  scenetime  conveytime  r ⋯
      │ Int64  Int64          Float64   Float64       Float64    Float64     F ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │     1             13       0.0       5.63333    19.9667         7.0    ⋯
    2 │     2             31       0.0       7.4        11.75          10.0
    3 │     3             25       0.0       6.55       11.75           9.0
    4 │     4             33       0.0       5.63333    13.2833         9.0
    5 │     5             25       0.0       6.48333    19.95           7.0    ⋯
    6 │     6              2       0.0       7.25       29.8833         8.0
    7 │     7             16       0.0       8.55       18.0667         9.0
    8 │     8             18       0.0       7.5        14.2            5.0
    9 │     9             27       0.0       8.15       17.9            7.0    ⋯
   10 │    10             13       0.0 

In [84]:
println(mean(df[!,"responsetime"]))
println(mean(df[!,"scenetime"]))
println(mean(df[!,"conveytime"]))
println(mean(df[!,"returntime"]))

6.690766666666667
16.000433333333337
7.106
7.268400000000001


## Temp results journal 

println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
println(mean(df[!,"returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>
#### log-log 3200 grid UNIFORM 44 WHOOPS
5.3978 <br>
16.000433333333337 <br>
15.4032 <br>
8.835816666666668 <br>
#### 3200 grid UNIFORM 44  WHOOPS
6.387766666666668 <br>
16.000433333333334 <br>
15.4032 <br>
8.805600000000002 <br>
#### 3200 grid stochastic
8.145316666666668
16.25061666666667
15.4032
9.455366666666668
#### old coefficients log-log 3200 grid stochastic 
5.700449999999999
16.385216666666665
6.282
5.874950000000001
#### new coefficients log-log 3200 grid stochastic
6.420166666666667
16.3654
7.106
6.756500000000001
#### v4 test calls ->
6.153766666666668
16.000433333333334
7.106
6.721716666666667
#### convey time looks too high try to change it to a static 15 minutes -> there was a /60 error again



both coefficients work.. which to use.

In [119]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
# include("..//src//problem.jl")
# model_name = "Robust01"

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#           num_regions,
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )

#     #testcalls
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")

#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     # #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df[!,"responsetime"]))

#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df[!,"responsetime"])
#     end
#     println(filename)
# end

"batch call simulation"

In [120]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

"all models simulation"

In [54]:
include("..//src//problem.jl")

Params(0.01, 0.5, 1.0e-6, 500, 50)

In [107]:
xsav = x

44-element Array{Int64,1}:
 1
 3
 1
 1
 0
 0
 1
 2
 1
 0
 0
 0
 0
 ⋮
 1
 2
 0
 1
 0
 0
 1
 0
 1
 0
 2
 1

In [121]:
"""batches for stochastic hype"""

for i = 1:3
    start_ind = (i-1)*1000+1
    end_ind = (i)*1000
    println("running $namb ambuances & $start_ind to $end_ind calls")
    test_calls_batch = test_calls[start_ind:end_ind,:]
    
    """Stochastic hype"""
    path = "austin_data_3200/stochastic_hyp"
    x_history = Any[]
    for j in readdir(path)
        x = readdlm(path * "/" * j, ',', Int64)
        x = reshape(x, (:))
        namb  = sum(x)
        push!(x_history,x)
        """ simulation parameters """
        p = DeploymentProblem(
              num_regions,
              hourly_calls,
              adjacent_nbhd,
              coverage,
              namb = namb,
              train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
              )
        #update 1: remove negative indices
        print(i)
        println("running $namb ambuances & $ncalls calls")
        #establishes the dispatch problem
        problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)
        # #establishes the dispatch model
        dispatch = ClosestDispatch(p, problem)

        df, guiArray = simulate_events!(problem, dispatch)
        print(mean(df[!,"responsetime"]))

        filename = string("batch",string(i),"_",string(j))
        open(filename, "w") do filename
            writedlm(filename,  df[!,"responsetime"])
        end
        println(filename)
    end    
    
end

running 45 ambuances & 1 to 1000 calls
1running 35 ambuances & 1000 calls
calling event id: 1 time: 38 value: 1012
closest available station : 1
143
243.0
Ambulance has arrived on the scene for event 1 at time 281
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 36
421
374.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance has arrived on the scene for event 3 at time 1047
calling event id: 5 time: 1166 value: 1621
closest available station : 14
375
357.0
calling event id: 6 time: 1187 value: 578
closest available station : 22
222
289.0
Ambulance for event 1 has arrived at the hospital at time 1265
Ambulance for event 1 is returning to the station at 1272
calling event id: 7 time: 1450 value: 1169
closest available station :

Ambulance has arrived on the scene for event 27 at time 8229
Ambulance for event 23 is returning to the station at 8230
Ambulance from event 24 has returned to station 17
Ambulance has arrived on the scene for event 29 at time 8371
Ambulance has arrived on the scene for event 30 at time 8471
calling event id: 31 time: 8588 value: 760
closest available station : 9
231
294.0
Ambulance from event 23 has returned to station 15
calling event id: 32 time: 8673 value: 1008
closest available station : 8
272
314.0
Ambulance for event 25 has arrived at the hospital at time 8679
Ambulance for event 25 is returning to the station at 8686
Ambulance has arrived on the scene for event 31 at time 8882
Ambulance for event 26 has arrived at the hospital at time 8916
Ambulance for event 30 has arrived at the hospital at time 8916
Ambulance for event 30 is returning to the station at 8922
Ambulance for event 26 is returning to the station at 8923
Ambulance has arrived on the scene for event 32 at time 898

calling event id: 55 time: 15714 value: 1437
closest available station : 15
378
358.0
Ambulance has arrived on the scene for event 53 at time 15716
calling event id: 56 time: 15831 value: 1664
closest available station : 34
475
392.0
Ambulance from event 50 has returned to station 14
Ambulance has arrived on the scene for event 54 at time 16057
Ambulance has arrived on the scene for event 55 at time 16072
Ambulance has arrived on the scene for event 56 at time 16223
calling event id: 57 time: 16247 value: 1009
closest available station : 3
230
294.0
Ambulance for event 53 has arrived at the hospital at time 16333
Ambulance for event 53 is returning to the station at 16338
Ambulance for event 51 has arrived at the hospital at time 16376
Ambulance for event 51 is returning to the station at 16384
Ambulance for event 52 has arrived at the hospital at time 16390
Ambulance for event 52 is returning to the station at 16396
Ambulance has arrived on the scene for event 57 at time 16541
Ambulan

Ambulance for event 77 has arrived at the hospital at time 25805
Ambulance for event 77 is returning to the station at 25813
Ambulance from event 76 has returned to station 14
calling event id: 79 time: 25937 value: 702
closest available station : 9
346
346.0
Ambulance from event 77 has returned to station 4
Ambulance has arrived on the scene for event 79 at time 26283
calling event id: 80 time: 26570 value: 1625
closest available station : 20
383
360.0
calling event id: 81 time: 26618 value: 1257
closest available station : 40
258
307.0
calling event id: 82 time: 26633 value: 1568
closest available station : 30
577
424.0
calling event id: 83 time: 26778 value: 1380
closest available station : 4
254
305.0
Ambulance for event 79 has arrived at the hospital at time 26869
Ambulance for event 79 is returning to the station at 26875
Ambulance has arrived on the scene for event 81 at time 26925
Ambulance for event 78 has arrived at the hospital at time 26927
Ambulance has arrived on the scen

Ambulance for event 102 has arrived at the hospital at time 37947
Ambulance for event 102 is returning to the station at 37954
Ambulance has arrived on the scene for event 104 at time 38118
calling event id: 105 time: 38194 value: 1683
closest available station : 14
347
346.0
Ambulance from event 102 has returned to station 9
calling event id: 106 time: 38341 value: 458
closest available station : 22
426
376.0
Ambulance for event 103 has arrived at the hospital at time 38373
Ambulance for event 103 is returning to the station at 38379
Ambulance has arrived on the scene for event 105 at time 38540
calling event id: 107 time: 38642 value: 1865
closest available station : 17
407
369.0
Ambulance has arrived on the scene for event 106 at time 38717
Ambulance from event 103 has returned to station 1
calling event id: 108 time: 38739 value: 1624
closest available station : 20
342
344.0
Ambulance for event 104 has arrived at the hospital at time 38848
Ambulance for event 104 is returning to th

...

calling event id: 524 time: 149123 value: 1500
closest available station : 14
426
376.0
calling event id: 525 time: 149140 value: 950
closest available station : 8
515
405.0
Ambulance has arrived on the scene for event 520 at time 149152
Ambulance has arrived on the scene for event 522 at time 149156
Ambulance has arrived on the scene for event 521 at time 149159
Ambulance for event 508 has arrived at the hospital at time 149233
Ambulance for event 508 is returning to the station at 149238
Ambulance from event 513 has returned to station 18
Ambulance for event 514 has arrived at the hospital at time 149312
Ambulance for event 514 is returning to the station at 149322
calling event id: 526 time: 149332 value: 643
closest available station : 8
648
444.0
Ambulance has arrived on the scene for event 523 at time 149420
calling event id: 527 time: 149472 value: 886
closest available station : 9
548
416.0
Ambulance has arrived on the scene for event 524 at time 149499
Ambulance has arrived on

Ambulance for event 540 is returning to the station at 153821
calling event id: 544 time: 153900 value: 1010
closest available station : 3
347
346.0
Ambulance from event 539 has returned to station 25
Ambulance has arrived on the scene for event 542 at time 154064
calling event id: 545 time: 154070 value: 1809
closest available station : 36
282
319.0
Ambulance has arrived on the scene for event 543 at time 154118
Ambulance from event 540 has returned to station 1
Ambulance has arrived on the scene for event 544 at time 154246
calling event id: 546 time: 154247 value: 880
closest available station : 18
468
390.0
Ambulance has arrived on the scene for event 545 at time 154389
Ambulance for event 541 has arrived at the hospital at time 154497
Ambulance for event 541 is returning to the station at 154502
calling event id: 547 time: 154599 value: 1253
closest available station : 24
472
391.0
calling event id: 548 time: 154627 value: 1071
closest available station : 1
224
290.0
Ambulance has

Ambulance from event 560 has returned to station 8
calling event id: 568 time: 160015 value: 1255
closest available station : 40
275
315.0
calling event id: 569 time: 160117 value: 1010
closest available station : 1
155
251.0
Ambulance from event 565 has returned to station 8
calling event id: 570 time: 160156 value: 1984
closest available station : 26
369
355.0
Ambulance from event 562 has returned to station 33
Ambulance for event 566 has arrived at the hospital at time 160210
Ambulance for event 566 is returning to the station at 160217
Ambulance has arrived on the scene for event 568 at time 160330
Ambulance has arrived on the scene for event 569 at time 160368
Ambulance has arrived on the scene for event 570 at time 160511
Ambulance from event 566 has returned to station 9
calling event id: 571 time: 160786 value: 1135
closest available station : 11
162
255.0
Ambulance has arrived on the scene for event 571 at time 161041
calling event id: 572 time: 161049 value: 1135
closest avai

Ambulance from event 590 has returned to station 25
Ambulance for event 589 has arrived at the hospital at time 168016
Ambulance for event 589 is returning to the station at 168024
calling event id: 595 time: 168056 value: 1622
closest available station : 25
236
297.0
Ambulance has arrived on the scene for event 593 at time 168219
Ambulance has arrived on the scene for event 594 at time 168228
Ambulance has arrived on the scene for event 595 at time 168353
Ambulance for event 588 has arrived at the hospital at time 168390
Ambulance for event 588 is returning to the station at 168398
Ambulance for event 591 has arrived at the hospital at time 168446

Excessive output truncated after 524316 bytes.

In [110]:
"""Stochastic hype"""
path = "austin_data_3200/stochastic_hyp"
x_history = Any[]
for i in readdir(path)
    repeat = false
    x = readdlm(path * "/" * i, ',', Int64)
    x = reshape(x, (:))
    namb  = sum(x)
    push!(x_history,x)
    count += 1;
    #if there are no repeats simulate.
    """ simulation parameters """
    if !(repeat)
        p = DeploymentProblem(
              num_regions,
              hourly_calls,
              adjacent_nbhd,
              coverage,
              namb = namb,
              train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
              )
        #update 1: remove negative indices
        test_calls = test_calls[1:ncalls,:]
        print(i)
        println("running $namb ambuances & $ncalls calls")
        
        #establishes the dispatch problem
        problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

        # #establishes the dispatch model
        
        dispatch = ClosestDispatch(p, problem)

        df, guiArray = simulate_events!(problem, dispatch)
        print(mean(df[!,"responsetime"]))

        filename = string(i)
        open(filename, "w") do filename
            writedlm(filename,  df[!,"responsetime"])
        end
        println(filename)
    end
    
end

stoch_35.csvrunning 35 ambuances & 1000 calls
calling event id: 1 time: 38 value: 1012
closest available station : 1
143
243.0
Ambulance has arrived on the scene for event 1 at time 281
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 36
421
374.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance has arrived on the scene for event 3 at time 1047
calling event id: 5 time: 1166 value: 1621
closest available station : 14
375
357.0
calling event id: 6 time: 1187 value: 578
closest available station : 22
222
289.0
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
513.0
Ambulance has arrived on the scene for event 6 at time 1476
Ambulance for event 1 has arrived at the hospital at time 1479
Ambulance fo

Ambulance has arrived on the scene for event 28 at time 8215
Ambulance has arrived on the scene for event 27 at time 8229
Ambulance has arrived on the scene for event 29 at time 8371
Ambulance from event 23 has returned to station 15
Ambulance has arrived on the scene for event 30 at time 8466
calling event id: 31 time: 8588 value: 760
closest available station : 9
231
294.0
Ambulance from event 25 has returned to station 22
calling event id: 32 time: 8673 value: 1008
closest available station : 8
272
314.0
Ambulance for event 26 has arrived at the hospital at time 8723
Ambulance for event 26 is returning to the station at 8730
Ambulance has arrived on the scene for event 31 at time 8882
Ambulance has arrived on the scene for event 32 at time 8987
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
344.0
Ambulance from event 26 has returned to station 20
Ambulance for event 28 has arrived at the hospital at time 9182
Ambulance for event 28 is returning to the s

346
346.0
Ambulance from event 77 has returned to station 4
Ambulance from event 76 has returned to station 14
Ambulance has arrived on the scene for event 79 at time 26283
Ambulance from event 78 has returned to station 4
calling event id: 80 time: 26570 value: 1625
closest available station : 20
383
360.0
calling event id: 81 time: 26618 value: 1257
closest available station : 40
258
307.0
calling event id: 82 time: 26633 value: 1568
closest available station : 30
577
424.0
calling event id: 83 time: 26778 value: 1380
closest available station : 4
254
305.0
Ambulance has arrived on the scene for event 81 at time 26925
Ambulance has arrived on the scene for event 80 at time 26930
Ambulance has arrived on the scene for event 82 at time 27057
Ambulance has arrived on the scene for event 83 at time 27083
Ambulance for event 79 has arrived at the hospital at time 27228
Ambulance for event 79 is returning to the station at 27234
calling event id: 84 time: 27471 value: 513
closest available

closest available station : 22
426
376.0
Ambulance has arrived on the scene for event 105 at time 38540
calling event id: 107 time: 38642 value: 1865
closest available station : 17
407
369.0
Ambulance has arrived on the scene for event 106 at time 38717
Ambulance for event 103 has arrived at the hospital at time 38723
Ambulance for event 103 is returning to the station at 38729
calling event id: 108 time: 38739 value: 1624
closest available station : 20
342
344.0
Ambulance for event 104 has arrived at the hospital at time 38821
Ambulance for event 104 is returning to the station at 38826
Ambulance has arrived on the scene for event 107 at time 39011
calling event id: 109 time: 39060 value: 1808
closest available station : 36
199
277.0
Ambulance from event 104 has returned to station 8
Ambulance has arrived on the scene for event 108 at time 39083
Ambulance from event 103 has returned to station 1
Ambulance has arrived on the scene for event 109 at time 39337
Ambulance for event 106 has

Ambulance from event 123 has returned to station 8
calling event id: 132 time: 47880 value: 1694
closest available station : 30
454
385.0
Ambulance for event 128 has arrived at the hospital at time 47961
Ambulance for event 128 is returning to the station at 47969
calling event id: 133 time: 48007 value: 883
closest available station : 8
559
419.0
Ambulance for event 129 has arrived at the hospital at time 48133
Ambulance for event 129 is returning to the station at 48141
Ambulance for event 130 has arrived at the hospital at time 48209
Ambulance for event 130 is returning to the station at 48215
Ambulance has arrived on the scene for event 132 at time 48265
Ambulance from event 128 has returned to station 26
Ambulance from event 129 has returned to station 8
Ambulance for event 131 has arrived at the hospital at time 48397
Ambulance for event 131 is returning to the station at 48404
Ambulance has arrived on the scene for event 133 at time 48426
Ambulance from event 130 has returned to

...

370
355.0
Ambulance has arrived on the scene for event 526 at time 149776
Ambulance has arrived on the scene for event 527 at time 149888
Ambulance for event 521 has arrived at the hospital at time 149936
Ambulance from event 517 has returned to station 34
Ambulance for event 521 is returning to the station at 149942
Ambulance from event 516 has returned to station 26
Ambulance has arrived on the scene for event 528 at time 150028
calling event id: 530 time: 150065 value: 1618
closest available station : 15
460
387.0
Ambulance has arrived on the scene for event 529 at time 150094
Ambulance from event 518 has returned to station 30
Ambulance for event 523 has arrived at the hospital at time 150279
Ambulance for event 523 is returning to the station at 150286
Ambulance from event 521 has returned to station 3
calling event id: 531 time: 150325 value: 1621
closest available station : 25
465
389.0
Ambulance for event 524 has arrived at the hospital at time 150415
Ambulance for event 524 is

Ambulance from event 542 has returned to station 17
Ambulance has arrived on the scene for event 549 at time 155371
calling event id: 551 time: 155424 value: 643
closest available station : 8
648
444.0
Ambulance from event 543 has returned to station 15
Ambulance has arrived on the scene for event 550 at time 155482
calling event id: 552 time: 155592 value: 1560
closest available station : 14
360
351.0
Ambulance from event 545 has returned to station 36
Ambulance for event 546 has arrived at the hospital at time 155768
Ambulance for event 546 is returning to the station at 155774
Ambulance has arrived on the scene for event 551 at time 155868
Ambulance for event 547 has arrived at the hospital at time 155942
Ambulance has arrived on the scene for event 552 at time 155943
Ambulance for event 547 is returning to the station at 155948
calling event id: 553 time: 155968 value: 1683
closest available station : 25
250
304.0
calling event id: 554 time: 155976 value: 1683
closest available sta

closest available station : 2
212
284.0
Ambulance has arrived on the scene for event 573 at time 161967
Ambulance has arrived on the scene for event 574 at time 161993
Ambulance from event 570 has returned to station 26
Ambulance for event 571 has arrived at the hospital at time 162112
Ambulance for event 571 is returning to the station at 162118
calling event id: 575 time: 162128 value: 1563
closest available station : 4
521
407.0
Ambulance for event 572 has arrived at the hospital at time 162257
Ambulance for event 572 is returning to the station at 162263
Ambulance for event 573 has arrived at the hospital at time 162455
Ambulance for event 573 is returning to the station at 162463
Ambulance from event 571 has returned to station 11
Ambulance from event 572 has returned to station 40
Ambulance has arrived on the scene for event 575 at time 162535
calling event id: 576 time: 162655 value: 2109
closest available station : 17
579
425.0
Ambulance for event 574 has arrived at the hospita

Excessive output truncated after 524340 bytes.


Ambulance from event 588 has returned to station 1
Ambulance has arrived on the scene for event 594 at time 168228
Ambulance for event 592 has arrived at the hospital at time 168273
Ambulance for event 592 is returning to the station at 168279


In [75]:
"""Robust cross validation"""
path = "austin_data_3200/cross_val_1"
x_history = Any[]
count = 1
for i in readdir(path)
    repeat = false
    x = readdlm(path * "/" * i, ',', Int64)
    x = reshape(x, (:))
    count2 = 1
    for j in x_history
        if x == j
            println(string(count) * "::" * string(count2))
            count2 += 1
            repeat = true
            break
        end
    end
    push!(x_history,x)
    count += 1;
    #if there are no repeats simulate.
    """ simulation parameters """
    if !(repeat)
        p = DeploymentProblem(
              num_regions,
              hourly_calls,
              adjacent_nbhd,
              coverage,
              namb = namb,
              train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
              )
        #update 1: remove negative indices
        test_calls = test_calls[1:ncalls,:]
        print(i)
        println("running $namb ambuances & $ncalls calls")
        
        #establishes the dispatch problem
        problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

        # #establishes the dispatch model
        
        dispatch = ClosestDispatch(p, problem)

        df, guiArray = simulate_events!(problem, dispatch)
        print(mean(df[!,"responsetime"]))

        filename = string(i)
        open(filename, "w") do filename
            writedlm(filename,  df[!,"responsetime"])
        end
        println(filename)
    end
    
end

cross1robust0.0001.csvrunning 40 ambuances & 1000 calls
calling event id: 1 time: 38 value: 1012
closest available station : 3
569
422.0
Ambulance has arrived on the scene for event 1 at time 460
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 25
477
393.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance has arrived on the scene for event 3 at time 1066
calling event id: 5 time: 1166 value: 1621
closest available station : 21
575
424.0
calling event id: 6 time: 1187 value: 578
closest available station : 22
222
289.0
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
513.0
Ambulance has arrived on the scene for event 6 at time 1476
calling event id: 8 time: 1578 value: 825
closest available stati

Ambulance has arrived on the scene for event 30 at time 8546
Ambulance for event 26 has arrived at the hospital at time 8581
calling event id: 31 time: 8588 value: 760
closest available station : 22
513
405.0
Ambulance for event 26 is returning to the station at 8588
calling event id: 32 time: 8673 value: 1008
closest available station : 18
856
497.0
Ambulance from event 25 has returned to station 22
Ambulance for event 27 has arrived at the hospital at time 8903
Ambulance for event 27 is returning to the station at 8912
Ambulance from event 24 has returned to station 25
Ambulance has arrived on the scene for event 31 at time 8993
Ambulance from event 26 has returned to station 21
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
344.0
Ambulance for event 30 has arrived at the hospital at time 9160
Ambulance for event 30 is returning to the station at 9166
Ambulance has arrived on the scene for event 32 at time 9170
Ambulance from event 27 has returned to sta

Ambulance has arrived on the scene for event 54 at time 16109
Ambulance from event 50 has returned to station 25
Ambulance for event 52 has arrived at the hospital at time 16176
Ambulance for event 52 is returning to the station at 16182
Ambulance has arrived on the scene for event 56 at time 16223
calling event id: 57 time: 16247 value: 1009
closest available station : 21
829
491.0
Ambulance from event 51 has returned to station 35
Ambulance from event 52 has returned to station 3
Ambulance for event 55 has arrived at the hospital at time 16661
Ambulance for event 55 is returning to the station at 16667
Ambulance has arrived on the scene for event 57 at time 16738
Ambulance for event 53 has arrived at the hospital at time 16757
Ambulance for event 53 is returning to the station at 16762
Ambulance for event 54 has arrived at the hospital at time 16973
Ambulance for event 54 is returning to the station at 16981
Ambulance from event 55 has returned to station 21
calling event id: 58 time

395
365.0
calling event id: 82 time: 26633 value: 1568
closest available station : 42
230
294.0
Ambulance from event 78 has returned to station 21
calling event id: 83 time: 26778 value: 1380
closest available station : 21
259
308.0
Ambulance has arrived on the scene for event 82 at time 26927
Ambulance has arrived on the scene for event 80 at time 26958
Ambulance has arrived on the scene for event 81 at time 26983
Ambulance has arrived on the scene for event 83 at time 27086
Ambulance for event 79 has arrived at the hospital at time 27277
Ambulance for event 79 is returning to the station at 27283
calling event id: 84 time: 27471 value: 513
closest available station : 2
93
204.0
Ambulance for event 82 has arrived at the hospital at time 27614
Ambulance for event 82 is returning to the station at 27623
Ambulance has arrived on the scene for event 84 at time 27675
Ambulance from event 79 has returned to station 22
calling event id: 85 time: 27753 value: 1010
closest available station : 

closest available station : 21
575
424.0
calling event id: 109 time: 39060 value: 1808
closest available station : 21
694
457.0
Ambulance has arrived on the scene for event 107 at time 39118
Ambulance has arrived on the scene for event 108 at time 39163
Ambulance for event 106 has arrived at the hospital at time 39272
Ambulance for event 106 is returning to the station at 39280
Ambulance for event 104 has arrived at the hospital at time 39310
Ambulance for event 104 is returning to the station at 39315
Ambulance for event 105 has arrived at the hospital at time 39401
Ambulance for event 105 is returning to the station at 39408
Ambulance has arrived on the scene for event 109 at time 39517
Ambulance for event 103 has arrived at the hospital at time 39543
Ambulance for event 103 is returning to the station at 39549
Ambulance from event 106 has returned to station 22
Ambulance from event 104 has returned to station 22
Ambulance from event 105 has returned to station 25
Ambulance for event

...

closest available station : 17
447
383.0
calling event id: 543 time: 153693 value: 1254
closest available station : 3
645
444.0
Ambulance has arrived on the scene for event 541 at time 153755
Ambulance from event 537 has returned to station 3
calling event id: 544 time: 153900 value: 1010
closest available station : 3
347
346.0
Ambulance has arrived on the scene for event 542 at time 154064
calling event id: 545 time: 154070 value: 1809
closest available station : 25
531
410.0
Ambulance has arrived on the scene for event 543 at time 154137
Ambulance for event 539 has arrived at the hospital at time 154139
Ambulance for event 539 is returning to the station at 154147
Ambulance for event 541 has arrived at the hospital at time 154178
Ambulance for event 541 is returning to the station at 154183
Ambulance has arrived on the scene for event 544 at time 154246
calling event id: 546 time: 154247 value: 880
closest available station : 18
468
390.0
Ambulance for event 540 has arrived at the ho

Ambulance has arrived on the scene for event 590 at time 167340
Ambulance has arrived on the scene for event 591 at time 167388
Ambulance for event 588 has arrived at the hospital at time 167421
Ambulance for event 588 is returning to the station at 167429
Ambulance from event 587 has returned to station 3
Ambulance for event 589 has arrived at the hospital at time 167523
Ambulance for event 589 is returning to the station at 167531
Ambulance has arrived on the scene for event 592 at time 167769
calling event id: 593 time: 167824 value: 834
closest available station : 44

Excessive output truncated after 524319 bytes.


484
395.0

LoadError: [91mInterruptException:[39m

In [29]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 1012, 1, 1, 1, -1, 0, DateTime("2021-07-19T12:39:09.102"))
 gui_event("call responded", 220, 31, 2, 31, -1, 0, DateTime("2021-07-19T12:39:09.196"))
 gui_event("call responded", 1687, 36, 3, 36, -1, 0, DateTime("2021-07-19T12:39:09.258"))
 gui_event("call responded", 1536, 33, 4, 33, -1, 0, DateTime("2021-07-19T12:39:09.304"))
 gui_event("call responded", 1621, 14, 5, 14, -1, 0, DateTime("2021-07-19T12:39:09.397"))
 gui_event("call responded", 578, 22, 6, 22, -1, 0, DateTime("2021-07-19T12:39:09.49"))
 gui_event("call responded", 1169, 16, 7, 16, -1, 0, DateTime("2021-07-19T12:39:09.505"))
 gui_event("call responded", 825, 8, 8, 8, -1, 0, DateTime("2021-07-19T12:39:09.721"))
 gui_event("call responded", 641, 9, 9, 9, -1, 0, DateTime("2021-07-19T12:39:09.8"))
 gui_event("call responded", 825, 3, 10, 3, -1, 0, DateTime("2021-07-19T12:39:09.892"))
 gui_event("call responded", 765, 35, 11, 35, -1, 0, DateTime("2021-07-19T12:39:09.985")

In [69]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"_3200grid_regression2",".csv")
open(filename, "w") do filename
    writedlm(filename,  df[!,"responsetime"])
end
print(filename)

visualization/time_adjust/Stochastic40calls1000_3200grid_regression2.csv

In [48]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")

In [4]:
readdir("C:/Users/Owner/Documents/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data_3200/cross_val_1")

LoadError: [91mSystemError: unable to read directory C:/Users/Owner/Documents/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/test/austin_data_3200/cross_val_1: Unknown error[39m